In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('./data/dogImages/train')
valid_files, valid_targets = load_dataset('./data/dogImages/valid')
test_files, test_targets = load_dataset('./data/dogImages/test')

# load list of dog names
dog_names = [item[20:-1] for item in sorted(glob("./data/dogImages/train/*/"))]

# print statistics about the dataset
print('There are %d total dog categories.' % len(dog_names))
print('There are %s total dog images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(299, 299))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# from keras.applications.mobilenet import MobileNet

# from keras.applications.resnet50 import ResNet50

In [ ]:
model = InceptionResNetV2(weights='imagenet', include_top=False, pooling=None)

# model = ResNet50(weights='imagenet', include_top=False, pooling=None)

# model = MobileNet(weights='imagenet', include_top=False, pooling=None, alpha=1.)

In [ ]:
train_InceptionResNetV2 = model.predict(train_tensors, batch_size=32, verbose=1)
valid_InceptionResNetV2 = model.predict(valid_tensors, batch_size=32, verbose=1)
test_InceptionResNetV2 = model.predict(test_tensors, batch_size=32, verbose=1)

In [ ]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Sequential

InceptionResNetV2_model = Sequential()
InceptionResNetV2_model.add(GlobalAveragePooling2D(input_shape=train_InceptionResNetV2.shape[1:]))
InceptionResNetV2_model.add(Dense(133, activation='softmax'))

InceptionResNetV2_model.summary()
### Define the model

### Compile the Model

In [ ]:
InceptionResNetV2_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
### Compile the model

### Train the Model

In [ ]:
# from keras.callbacks import ModelCheckpoint  

# checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.InceptionResNetV2_model.hdf5', 
#                                verbose=1, save_best_only=True)

# InceptionResNetV2_model.fit(train_InceptionResNetV2, train_targets, 
#           validation_data=(valid_InceptionResNetV2, valid_targets),
#           epochs=20, batch_size=32, callbacks=[checkpointer], verbose=1)
# ### Train the model.

### Load the Model with the Best Validation Loss

In [ ]:
InceptionResNetV2_model.load_weights('saved_models/weights.best.InceptionResNetV2_model.hdf5')
### Load the model weights with the best validation loss.VGG16_model.load_weights('saved_models/weights.best.VGG16.hdf5')

### Test the Model

Try out your model on the test dataset of dog images. Ensure that your test accuracy is greater than 60%.

In [ ]:
# get index of predicted dog breed for each image in test set
InceptionResNetV2_predictions = [np.argmax(InceptionResNetV2_model.predict(np.expand_dims(feature, axis=0))) for feature in test_InceptionResNetV2]

# report test accuracy
test_accuracy = 100*np.sum(np.array(InceptionResNetV2_predictions)==np.argmax(test_targets, axis=1))/len(InceptionResNetV2_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)
### Calculate classification accuracy on the test dataset.

In [ ]:
InceptionResNetV2_model.layers

In [ ]:
from keras.models import Model, Input

inp_ten = Input(shape=(299, 299, 3))
x = InceptionResNetV2(weights='imagenet', include_top=False, pooling=None)(inp_ten)
x = InceptionResNetV2_model.layers[0](x)
out = InceptionResNetV2_model.layers[1](x)

model_m = Model(inp_ten, out)

In [ ]:
model_m.summary()

In [ ]:
# get index of predicted dog breed for each image in test set
InceptionResNetV2_predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(InceptionResNetV2_predictions)==np.argmax(test_targets, axis=1))/len(InceptionResNetV2_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)
### Calculate classification accuracy on the test dataset.

In [ ]:
model_m.save_weights('../input/dog_identification_model.hdf5')

In [ ]:
from keras.models import Model, Input

inp_ten = Input(shape=(299, 299, 3))
x = InceptionResNetV2(weights='imagenet', include_top=False, pooling=None)(inp_ten)
x = GlobalAveragePooling2D()(x)
out = Dense(133, activation='softmax')(x)

model = Model(inp_ten, out)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

model.load_weights('../input/dog_identification_model.hdf5')

In [ ]:
model.load_weights('../input/dog_identification_model.hdf5')

In [ ]:
import pandas as pd


labels = pd.read_csv('../input/petfinder-adoption-prediction/breed_labels.csv')
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')

In [ ]:
def a(x):
    b1 = x['Breed1']
    
    if b1 == 307:
        b2 = x['Breed2']
        if b2 % 307 != 0:
            return b2
    
    return b1

train['Breed1'] = train.apply(a, axis=1)
test['Breed1'] = test.apply(a, axis=1)

In [ ]:
mixed_types_train = train[(train.Type==1) & (train.Breed1 == 307)][['PetID', 'Breed1', 'Breed2']]
mixed_types_test = test[(test.Type==1) & (test.Breed1 == 307)][['PetID', 'Breed1', 'Breed2']]

In [ ]:
import cv2

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
breed_dict = {i:j for i, j in zip(labels[labels.Type == 1].BreedName.apply(lambda x: x.lower().replace(' ', '_')).values, labels[labels.Type == 1].BreedID.values)}

In [ ]:
mapping = {
    'american_foxhound': 'foxhound',
    'anatolian_shepherd_dog': 'anatolian_shepherd',
    'australian_cattle_dog': 'australian_cattle_dog/blue_heeler',
    'belgian_malinois': 'belgian_shepherd_malinois',
    'belgian_sheepdog': 'belgian_shepherd_dog_sheepdog',
    'belgian_tervuren': 'belgian_shepherd_tervuren',
    'bouvier_des_flandres': 'bouvier_des_flanders',
    'brittany': 'brittany_spaniel',
    'bulldog': 'american_bulldog',
    'cane_corso': 'cane_corso_mastiff',
    'cardigan_welsh_corgi': 'welsh_corgi',
    'chinese_crested': 'chinese_crested_dog',
    'chinese_shar-pei': 'shar_pei',
    'dandie_dinmont_terrier': 'dandi_dinmont_terrier',
    'entlebucher_mountain_dog': 'entlebucher',
    'icelandic_sheepdog': 'shetland_sheepdog_sheltie',
    'irish_red_and_white_setter': 'irish_setter',
    'miniature_schnauzer': 'schnauzer',
    'newfoundland': 'newfoundland_dog',
    'nova_scotia_duck_tolling_retriever': 'nova_scotia_duck-tolling_retriever',
    'parson_russell_terrier': 'jack_russell_terrier',
    'pembroke_welsh_corgi': 'welsh_corgi',
    'plott': 'plott_hound',
    'wirehaired_pointing_griffon': 'german_wirehaired_pointer',
    'xoloitzcuintli': 'xoloitzcuintle/mexican_hairless',
    'yorkshire_terrier': 'yorkshire_terrier_yorkie'
}

### A function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.
img_path = '../input/petfinder-adoption-prediction/train_images/ef14861df-1.jpg'
#img_path = dog_files_short[2]
def classify_dog_breed(img_path):
    img = path_to_tensor(img_path).astype('float32')/255
    predictions = model.predict(img)
    prediction = np.argmax(predictions)
    new_dog_names = dog_names[prediction].split('.')[-1]
    dog_name = new_dog_names.lower()
    dog_name = dog_name if dog_name not in mapping else mapping[dog_name]
    return [predictions.max(), dog_name, breed_dict[dog_name]]
img = cv2.imread(img_path)
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
prediction = classify_dog_breed(img_path)
prediction

In [ ]:
dog_names

In [ ]:
import os

def get_type(x):
    img_path = f'../input/petfinder-adoption-prediction/train_images/{x}-1.jpg'
    if os.path.exists(img_path):
        return classify_dog_breed(img_path)
    return [307, '', 0]

def get_type_test(x):
    img_path = f'../input/petfinder-adoption-prediction/test_images/{x}-1.jpg'
    if os.path.exists(img_path):
        return classify_dog_breed(img_path)
    return [307, '', 0]

mixed_types_train['prob'] = 0
mixed_types_train['name'] = 0
mixed_types_train['idd'] = 0

mixed_types_test['prob'] = 0
mixed_types_test['name'] = 0
mixed_types_test['idd'] = 0

In [ ]:
mix_train = mixed_types_train.PetID.apply(get_type).values
mix_test = mixed_types_test.PetID.apply(get_type_test).values

In [ ]:
mixed_types_train.loc[:, ['prob', 'name', 'idd']] = list(mix_train)
mixed_types_test.loc[:, ['prob', 'name', 'idd']] = list(mix_test)

In [ ]:
mixed_types_train.to_csv('../input/mixed_breed_train.csv', index=0)
mixed_types_test.to_csv('../input/mixed_breed_test.csv', index=0)